In [ ]:
#v.20251106
#b.maier
#for calibrating gsi to fems field sample fuel moisture data
#most of gsi calculation script lifted from matt jolly

import requests
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import urllib
import matplotlib.dates as mdates
import os.path
from math import *
import itertools
#%pylab inline
import matplotlib.pyplot as plt
import warnings
from datetime import datetime, timezone, timedelta

proj_crs = 4326
tz_local = 'US/Pacific'

cwd = 'C:\\1_Projects\\NfdrsGsi\\'

In [ ]:
#fems api urls
url_fuelmodel = 'https://fems.fs2c.usda.gov/fuelmodel/apis/graphql'
url_climatology = 'https://fems.fs2c.usda.gov/api/climatology/graphql'

#func to return data from fems api
def returnGraphQl(url, query, variables):
	r = requests.post(url, json={'query': query, 'variables': variables})
	r_json = json.loads(r.text)
	return r_json

#for making output dirs
import os
def make_dir(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    else:
        pass
make_dir(cwd)

In [ ]:
#return geojson from rest service
#set to use pnw fdra, if using other fdra here fdra must have attributes ParentDocName and FDRAName... 
def geojson_rest(url):
	req = urllib.request.urlopen(url)
	gjson = req.read().decode('utf-8')
	gdf = gpd.read_file(gjson)
	return gdf 

#get pnw fdra
def get_fdra():
    print("Getting Fdra...")
    url = 'https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/Pacific_Northwest_Fire_Danger_Rating_Areas_(public)/FeatureServer/0/query?where=OBJECTID+%3E+-1&outFields=FDRAName,ParentDocName,PythonURL&f=pgeojson&token='
    gdf = geojson_rest(url).to_crs(proj_crs)
    return gdf
gdf_fdra = get_fdra()

gdf_fdra['key'] = gdf_fdra['ParentDocName'].str.replace('\W', '', regex=True) + '_' + gdf_fdra['FDRAName'].str.replace('\W', '', regex=True)

In [ ]:
#class for retrieving fems data
class FemsFieldSample(object):
    def __init__(self, gdf_select):
        self.gdf_select = gdf_select
        self.sites = None
        self.site_ids = None
        self.samples = None
        self.fuels = None
        self.gdf_samples = None

    def get_sites(self):
        #get field sample sites
        fm_site_query = """
        query GetSites {
            getSites(returnAll: true) {
                sites {
                    siteId
                    siteName
                    areaId
                    areaName
                    stateId
                    stateName
                    groupId
                    groupName
                    siteStartDate
                    active
                    siteStatus
                    fuelModel
                    fuelLoading
                    slope
                    aspect
                    defaultEndMonth
                    defaultBeginMonth
                    remarks
                    rawsId
                    raws
                    jurisdiction
                    zoomLevel
                    longitude
                    latitude
                    elevation
                    timeZone
                    timeZoneOffset
                    mapWidth
                    modifiedTime
                    modifiedBy
                    createdTime
                    createdBy
                    firstSampleDate
                    latestSampleDate
                }
            }
        }
        """
        print("Getting Sites...")
        json = returnGraphQl(url_fuelmodel, fm_site_query, None)
        df = pd.DataFrame(json['data']['getSites']['sites'])
        gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['longitude'], df['latitude'])).set_crs(proj_crs)
        gdf = gpd.clip(gdf, self.gdf_select)
        self.sites = gdf
        self.site_ids = [", ".join(map(str, gdf.siteId.unique()))][0]

    #get field sample samples
    def get_samples(self):
        
        fm_sample_query = """
        query GetFuelSamples ($siteId: String!) {
            getFuelSamples(
                returnAll: false
                sampleId: null
                siteId: $siteId
                startDate: "1910-01-01T00:00+00"
                endDate: "2050-01-01T00:00+00"
                filterBySampleType: "All"
                filterByStatus: "All"
                filterByCategory: "All"
                filterBySubCategory: "All"
                filterByMethod: "All"
            ) {
                fuelSamples {
                    fuel_sample_id
                    site_id
                    fuel_id
                    sub_category_id
                    sub_category
                    sample
                    method_type
                    status
                    sample_average_value
                    subSampleCount
                    modified_time
                    modified_by
                    created_time
                    created_by
                }
            }
        }
        """
        print("Getting Samples...")
        json = returnGraphQl(url_fuelmodel, fm_sample_query, {'siteId': self.site_ids})
        df = pd.DataFrame(json['data']['getFuelSamples']['fuelSamples'])
        df['datetime'] = pd.to_datetime(df['sample'], errors='coerce')
        df['doy'] = df.datetime.dt.dayofyear
        df = df.loc[(df.status == 'Submitted') | (df.status == 'Edited')]
        self.samples = df

    def get_fuels(self):
        #get field sample fuels
        fuels_sample_query = """
        query GetFuels {
            getFuels(returnAll: true) {
                fuel_id
                category
                fuel_type
                scientific_name
            }
        }
        """
        print("Getting Fuels...")
        json = returnGraphQl(url_fuelmodel, fuels_sample_query, None)
        df = pd.DataFrame(json['data']['getFuels'])
        self.fuels = df

    def add_fuels_to_samples(self):
        #add fuels data to samples data
        self.samples['category'] = ''
        self.samples['fuel_type'] = ''
        self.samples['scientific_name'] = ''
        for i, row in self.samples.iterrows():
            r_fuel_id = self.samples.loc[i, 'fuel_id']
            r_fuels = self.fuels.loc[self.fuels.fuel_id == r_fuel_id]
            self.samples.loc[i, 'category'] = r_fuels['category'].values[0]
            self.samples.loc[i, 'fuel_type'] = r_fuels['fuel_type'].values[0]
            self.samples.loc[i, 'scientific_name'] = r_fuels['scientific_name'].values[0]

    def make_gdf_samples(self):
        df = self.sites.set_index('siteId').join(self.samples.set_index('site_id')) #note fems site id are currently different...
        self.gdf_samples = df

In [ ]:
class AnalysisData(object):   
    
    def __init__(self, gdf_select):
        self.gdf_select = gdf_select
        self.field_sample = FemsFieldSample(gdf_select)
        self.sites = None
        self.samples = None
        self.fuels = None
        self.raws = None
        
    def get_field_sample(self):
        self.sites = self.field_sample.get_sites()
        self.samples = self.field_sample.get_samples()
        self.fuels = self.field_sample.get_fuels()
        self.field_sample.add_fuels_to_samples()
        self.field_sample.make_gdf_samples()

    #get raws from wxx
    def get_raws(self):
        print("Getting Raws...")
        url_wxx = 'https://weather.nifc.gov/ords/prd/wx/station/statbaseline/0'
        with urllib.request.urlopen(url_wxx) as url:
            data = json.load(url)
        df = pd.DataFrame(data['station_archive_baseline'])
        df = df.loc[df.Class == 'Permanent']
        df = df.loc[df.Status == 'A']
        df = df.loc[df['Ownership Type'] == 'FIRE']
        #remove leading zeros from station id, fems does not recognize (station id is an int in fems, not a str)
        df['NWS ID'] = df['NWS ID'].str.lstrip('0')
        gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs=proj_crs)
        self.raws = gdf   

In [ ]:
gdf_fdra.ParentDocName.unique()

In [ ]:
#general analysis area of interest, script is setup for this to be fdop level
#can also just load up the whole gacc here
gdf_select_aoi = gdf_fdra#.loc[gdf_fdra.ParentDocName == 'Southwest Oregon']
analysis_data_all = AnalysisData(gdf_select_aoi)
analysis_data_all.get_field_sample()
analysis_data_all.get_raws()

In [ ]:
class GsiAnalysisData(object):

    def get_fems_wx(sta_id): #sta_id is nws_id, fems was going away from nws_id but now not so much...
        wx_query = """
        query WeatherObs ($stationIds: String!) {
            weatherObs(
                startDateTimeRange: "2005-01-01T00:00:00Z"
                endDateTimeRange: "2025-12-31T00:00:00Z"
                stationIds: $stationIds
            ) {
                data {
                    station_id
                    station_name
                    latitude
                    longitude
                    elevation
                    zoom_level
                    station_type
                    observation_time
                    observation_time_offset
                    display_hour
                    display_hour_offset
                    display_date
                    temperature
                    relative_humidity
                    hourly_precip
                    hr24Precipitation
                    hr48Precipitation
                    hr72Precipitation
                    wind_speed
                    wind_direction
                    peak_gust_speed
                    peak_gust_dir
                    sol_rad
                    snow_flag
                    observation_type
                    hex
                    t_flag
                    rh_flag
                    pcp_flag
                    ws_flag
                    wa_flag
                    sr_flag
                    gs_flag
                    ga_flag
                }
            }
        }
        """
        json_wx = returnGraphQl(url_climatology, wx_query, {"stationIds": sta_id})
        df = pd.DataFrame(json_wx['data']['weatherObs']['data'])
        df.observation_time = pd.to_datetime(df.observation_time).dt.tz_convert(tz_local)
        df = df.loc[df.observation_type=='O']
        df['doy'] = df.observation_time.dt.dayofyear
        return df

    def __init__(self, gdf_select, analysis_data_all):
        self.analysis_data_all = analysis_data_all
        self.gdf_select = gdf_select
        self.fdop_name = gdf_select.ParentDocName.values[0]
        self.fdra_name = gdf_select.FDRAName.values[0]
        self.fdop_dir = ''.join(char for char in self.fdop_name if char.isalnum())
        self.fdra_dir = ''.join(char for char in self.fdra_name if char.isalnum())
        self.raws = None # gpd.clip(analysis_data_all.field_sample.gdf_samples
        self.sites = None
        self.samples_herb = None
        self.samples_shrub = None
        self.samples_herb_group = None
        self.samples_shrub_group = None
        self.wx = None

    def clip_data(self):
        self.raws = gpd.clip(self.analysis_data_all.raws, self.gdf_select)
        self.sites = gpd.clip(self.analysis_data_all.field_sample.gdf_samples, self.gdf_select)

    def prep_data(self):
        self.samples_shrub = self.sites.loc[(self.sites.category == 'Shrub')]
        self.samples_herb = self.sites.loc[(self.sites.category == 'Grass') | (self.sites.category == 'Forb')]
        self.samples_shrub_group = self.samples_shrub.groupby('doy')[['sample_average_value']].mean()
        self.samples_herb_group = self.samples_herb.groupby('doy')[['sample_average_value']].mean()
        self.samples_shrub_group['rolling_mean'] = self.samples_shrub_group.sample_average_value.rolling(10, center=True, min_periods=5).mean()
        self.samples_herb_group['rolling_mean'] = self.samples_herb_group.sample_average_value.rolling(10, center=True, min_periods=5).mean()
        #remove crazy values
        self.samples_shrub_group.loc[self.samples_shrub_group.rolling_mean > 500] = np.nan
        self.samples_herb_group.loc[self.samples_herb_group.rolling_mean > 500] = np.nan 

    def get_wx(self):
        l = []
        for sta_id in self.raws['NWS ID'].unique():
            sta_id = str(sta_id)
            try:
                print("Getting " + sta_id + " Wx...")
                df = GsiAnalysisData.get_fems_wx(sta_id)
                l.append(df)
            except Exception as e:
                print("Station " + sta_id + " Failed...")
        self.wx = pd.concat(l, axis=0)

In [ ]:
#plot fdra field sample fuels and mean (calibration target)
from cycler import cycler
def plot_analysis_area_fuels(analysis_data):
    output_dir = cwd + analysis_data.fdop_dir
    make_dir(cwd + analysis_data.fdop_dir)
    plt.rcParams['axes.prop_cycle'] = cycler('color', plt.get_cmap('tab20c_r').colors)
    shrub = analysis_data.samples_shrub.copy()
    herb = analysis_data.samples_herb.copy()
    
    shrub_nyears = len(shrub.datetime.dt.year.unique())
    herb_nyears = len(herb.datetime.dt.year.unique())
    shrub_grp = analysis_data.samples_shrub_group.copy()
    herb_grp = analysis_data.samples_herb_group.copy()
    shrub_grp_sp = shrub.groupby(['fuel_type', 'doy'])[['sample_average_value']].mean().unstack().T \
        .reset_index().drop(['level_0'], axis=1).set_index('doy', drop=True)
    herb_grp_sp = herb.groupby(['fuel_type', 'doy'])[['sample_average_value']].mean().unstack().T \
        .reset_index().drop(['level_0'], axis=1).set_index('doy', drop=True)
    fig, (ax1, ax2) = plt.subplots(2, figsize=(4,5.5))
    if len(shrub_grp_sp.columns) == 1:
        label = shrub_grp_sp.columns[0]
    else:
        label = shrub_grp_sp.columns
    ax1.plot(shrub_grp_sp, linewidth=0.5, label=label)
    ax1.plot(shrub_grp.index, shrub_grp.sample_average_value, color='black', label='Sample Mean')
    ax1.plot(shrub_grp.index, shrub_grp.rolling_mean, color='red', label='Sample Rolling Mean')
    ax1.set_xlim(0, 365)
    ax1.set_ylim(0, 250)
    ax1.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
    ax1.set_title('Shrub:' + ' Years(n) = ' + str(shrub_nyears))
    ax1.legend(prop={'size': 6})
    if len(herb_grp_sp.columns) == 1:
        label = herb_grp_sp.columns[0]
    else:
        label = herb_grp_sp.columns
    ax2.plot(herb_grp_sp, linewidth=0.5, label=label)
    ax2.plot(herb_grp.index, herb_grp.sample_average_value, color='black', label='Sample Mean')
    ax2.plot(herb_grp.index, herb_grp.rolling_mean, color='red', label='Sample Rolling Mean')
    ax2.set_xlim(0, 365)
    ax2.set_ylim(0, 250)
    ax2.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
    ax2.set_title('Grass & Forb:' + ' Years(n) = ' + str(herb_nyears))
    ax2.legend(prop={'size': 6})
    plt.suptitle(analysis_data.fdop_name + ' ' + analysis_data.fdra_name)
    plt.tight_layout()
    fig.savefig(output_dir + '\\samples_' + analysis_data.fdop_dir + '_' + analysis_data.fdra_dir + '.jpg')
    plt.close()

In [ ]:
#plot field sample data to see if enough data for gsi calibration
def make_field_sample_plots(gdf_fdra):
    for fdra_key in gdf_fdra.key.unique():
        gdf_select_fdra = gdf_fdra.loc[gdf_fdra.key == fdra_key]
        analysis_data = GsiAnalysisData(gdf_select_fdra, analysis_data_all)
        analysis_data.clip_data()
        analysis_data.prep_data()
        if analysis_data.sites.empty != True:
            plot_analysis_area_fuels(analysis_data)
make_field_sample_plots(gdf_fdra)

In [ ]:
def calibrate_gsi(analysis_data, herb_or_woody):

    #plot_analysis_area_fuels(field_sample)

    if not use_prcp:
    	fname_ext = '_VpdMax'
    if use_prcp:
    	fname_ext = '_Prcp'

    analysis_data_wx = analysis_data.wx.copy()
    analysis_data_raws = analysis_data.raws.copy()
    samples_shrub = analysis_data.samples_shrub_group.copy()
    samples_herb = analysis_data.samples_herb_group.copy()
    output_dir = cwd + analysis_data.fdop_dir + '\\' + analysis_data.fdra_dir
    make_dir(output_dir)

    if herb_or_woody == 'Woody':
        samples = samples_shrub.copy()
    elif herb_or_woody == 'Herbaceous':
        samples = samples_herb.copy()
    else:
        print("Options are Woody or Herbaceous...")
        raise ValueError

    sta_results_list = []
    for sta_id in analysis_data_wx.station_id.unique():
        print("Calibrating Gsi " + str(sta_id) + "...")
        df_wx = analysis_data_wx.loc[analysis_data_wx.station_id == sta_id, :].copy()
        sta_id = str(sta_id)
        st_yr = str(df_wx.observation_time.dt.year.values.min())
        end_yr = str(df_wx.observation_time.dt.year.values.max())
        chart_yrs = "(" + st_yr + "-" + end_yr + ")"

        #---Most of the following was lifted from from Matt Jolly's Git NFDR Master Class before alteration---#
        # This function will calcuraws_late Photoperiod from raws_latitude and Day of Year
        RADPERDAY = 0.017214
        RADPERDEG = 0.01745329
        MINDECL = -0.4092797
        SECPERRAD = 13750.9871
        DAYSOFF = 10.25
        #Inputs: raws_lat = raws_latitude in Degrees, yday is the day of the year.
        #Returns: Daylength in seconds
        def CalcDayl(raws_lat, julian_day):
                # Daylength function from MT-CLIM */
                raws_lat = raws_lat * RADPERDEG
                if raws_lat > 1.5707:
                    raws_lat = 1.5707
                if raws_lat < -1.5707:
                    raws_lat = -1.5707
                cosraws_lat = cos(raws_lat)
                sinraws_lat = sin(raws_lat)

                #* calcuraws_late cos and sin of declination */
                decl = MINDECL * cos((julian_day + DAYSOFF) * RADPERDAY)
                cosdecl = cos(decl)
                sindecl = sin(decl)
                cosegeom = cosraws_lat * cosdecl
                sinegeom = sinraws_lat * sindecl
                coshss = -(sinegeom) / cosegeom
                if coshss < -1.0:
                    coshss = -1.0  # 24-hr daylight */
                if coshss > 1.0:
                    coshss = 1.0    # 0-hr daylight */
                hss = acos(coshss)                # hour angle at sunset (radians) */
                #* daylength (seconds) */
                return 2.0 * hss * SECPERRAD
        # Calcuate Daylength for each row                          
        df_wx['Dayl'] = df_wx.apply(lambda row: CalcDayl(df_wx.latitude.round(0)[0], int(row['doy'])), axis=1)

        # Function to calculate the Saturation Vapor Pressure for a given temperature.
        # Note: Function converts internally from F to C.
        def CalcVP(tempF):
            tmpC =  (tempF - 32.0) / 1.8
            vp = 610.7 * exp((17.38 * tmpC)/(239 + tmpC))
            return vp

        # Calculate the VPD from RH and temperature
        def CalcVPD(RH, TempF):
            vp = CalcVP(TempF)
            vpd = vp - (RH / 100) * vp
            if(vpd < 0.0):
                vpd = 0.0;
            return vpd

        # Use of resample requires datetime as index
        df_wx_hourly = df_wx.set_index('observation_time')

        Tmin = pd.DataFrame({"Tmin": df_wx_hourly['temperature'].resample('D').min()})  
        Tmax = pd.DataFrame({"Tmax": df_wx_hourly['temperature'].resample('D').max()})
        Tavg = pd.DataFrame({"Tavg": df_wx_hourly['temperature'].resample('D').mean()})
        Dayl = pd.DataFrame({"Dayl": df_wx_hourly['Dayl'].resample('D').mean()})
        Prcp = pd.DataFrame({"Prcp": df_wx_hourly['hourly_precip'].resample('D').sum()})
        RHmin = pd.DataFrame({"RHmin": df_wx_hourly['relative_humidity'].resample('D').min()})
        RHmax = pd.DataFrame({"RHmax": df_wx_hourly['relative_humidity'].resample('D').max()})
        RHavg = pd.DataFrame({"RHavg": df_wx_hourly['relative_humidity'].resample('D').mean()})

        df_wx_daily = pd.concat([Tmax, Tmin, Tavg, RHmax, RHmin, RHavg, Dayl, Prcp], axis=1)
        df_wx_daily['observation_time'] = df_wx_daily.index
        df_wx_daily['VPDAvg'] = df_wx_daily.apply(lambda row: CalcVPD(row['RHavg'], row['Tavg']),axis=1)
        df_wx_daily['VPDMax'] = df_wx_daily.apply(lambda row: CalcVPD(row['RHmin'], row['Tmax']),axis=1)
        df_wx_daily['TminC'] =  (df_wx_daily.Tmin - 32.0) * 5.0 / 9.0; # Convert Tmin from Fahrenheit to celcuius
        df_wx_daily['Prcp_RT'] = df_wx_daily['Prcp'].rolling(28).sum()

        # GSI indicator/ramp function
        #change from mj version to use arrays instead of apply by row (slow)
        def Ind(Var, Low, Up):
            with np.errstate(divide='ignore'): #ignore divide by zero runtimewarning
                arr = (Var - Low) / (Up - Low) #return the proportion
                arr[np.isinf(arr)] = 0 #replace inf from divide by zero with 0
            arr[Var > Up] = 1 #if the variable is greater than the upper limit, return 1
            arr[Var < Low] = 0 #if the variable is less than the lower limit, return 0
            arr[Up == Low] = 0 #upper (Up) and lower (Low) can't be the same
            return arr

        def make_setting_combinations(use_prcp):
            #use station values for ranges
            t_min = round(df_wx_daily['TminC'].min() / 2) * 2
            if t_min < -10:
                t_min = -10
            t_max = round(df_wx_daily['TminC'].max() / 2) * 2
            if t_max > 22:
                t_max = 22

            if use_vpdmax:
                vpd_min = round(df_wx_daily.VPDMax.min() / 50) * 50
                if vpd_min < 250:
                    vpd_min = 0
                else:
                    vpd_min = round(df_wx_daily.VPDMax.min() / 250) * 250
                vpd_max = round(df_wx_daily.VPDMax.max() / 250) * 250
            else:
                vpd_min = round(df_wx_daily.VPDAvg.min() / 50) * 50
                if vpd_min < 250:
                    vpd_min = 0
                else:
                    vpd_min = round(df_wx_daily.VPDAvg.min() / 250) * 250
                vpd_max = round(df_wx_daily.VPDAvg.max() / 250) * 250                    
            
            dayl_min = int(df_wx_daily.Dayl.min() / 3600) * 3600
            dayl_max = int(df_wx_daily.Dayl.max() / 3600) * 3600        
                
            #careful here, easy to get to a million possible combinations pretty quickly
            TminLowRange = np.arange(t_min, 0, 2) #-10
            TminHighRange = np.arange(0, t_max, 2) #25
            VPDLowRange = np.arange(vpd_min, 3500, 250) #0
            VPDUpRange = np.arange(3500, vpd_max, 250) #10000
            DaylLowRange = np.arange(dayl_min, dayl_max, 3600)
            DaylUpRange = np.arange(dayl_min + 3600, dayl_max + 3600, 3600)
            if use_prcp:
                PrcpRTLow = [0.25, 0.5, 0.75, 1.0]
                PrcpRTUp = [1.25, 1.5, 2.0, 2.25]
                combinations = np.array(list(itertools.product(TminLowRange, TminHighRange, VPDLowRange, \
                    VPDUpRange, DaylLowRange, DaylUpRange, PrcpRTLow, PrcpRTUp)))
            else:
                combinations = np.array(list(itertools.product(TminLowRange, TminHighRange, VPDLowRange, \
                    VPDUpRange, DaylLowRange, DaylUpRange)))
            return combinations

        if herb_or_woody == 'Woody':
            LFMMax = 250 #samples.sample_average_value.max()
            if samples.sample_average_value.min() >= 60:
                LFMMin = samples.sample_average_value.min()
            else:
                LFMMin = 60
        else:
            LFMMax = 300 #samples.sample_average_value.max()
            if samples.sample_average_value.min() >= 30:
                LFMMin = samples.sample_average_value.min()
            else:
                LFMMin = 30            

        if use_prcp:
            GUThresh = 0.3
        else:
            GUThresh = 0.5

        samples.rename({'rolling_mean': 'FuelMoisture'}, axis=1, inplace=True)

        def calibrate_gsi(df_wx_daily, samples, use_vpdmax, use_prcp):
            df = df_wx_daily.copy()
            combinations = make_setting_combinations(use_prcp)
            combo_list = []
            combo_len = len(combinations)	
            arr_len = len(df)

            m = (LFMMax - LFMMin) / (1 - GUThresh)
            m_arr = np.repeat(m, arr_len)
            b = LFMMax - m
            b_arr = np.repeat(b, arr_len)
            guthresh_arr = np.repeat(GUThresh, arr_len)

            for combination in combinations:
                try:
                    tmin_arr = Ind(df.TminC.values, \
                        np.repeat(combination[0], arr_len), np.repeat(combination[1], arr_len))		
                    df['TminInd'] = tmin_arr.tolist()

                    if use_vpdmax:
                        vpd_arr = Ind(df.VPDMax.values, \
                            np.repeat(combination[2], arr_len), np.repeat(combination[3], arr_len))
                    else:
                        vpd_arr = Ind(df.VPDAvg.values, \
                            np.repeat(combination[2], arr_len), np.repeat(combination[3], arr_len))
                    df['VPDInd'] = vpd_arr.tolist()
                    df['VPDInd'] = 1 - df['VPDInd']

                    dayl_arr = Ind(df.Dayl.values, \
                        np.repeat(combination[4], arr_len), np.repeat(combination[5], arr_len))	
                    df['DaylInd'] = dayl_arr.tolist()

                    if use_prcp:
                        prcp_arr = Ind(df.Prcp_RT.values, \
                            np.repeat(combination[6], arr_len), np.repeat(combination[7], arr_len))
                        df['PrcpInd'] = prcp_arr.tolist()
                        df['iGSI'] = df['TminInd'] * df['DaylInd'] * df['VPDInd'] * df['PrcpInd']
                    else:
                        df['iGSI'] = df['TminInd'] * df['DaylInd'] * df['VPDInd']	

                    df['GSI'] = df['iGSI'].rolling(28).mean()

                    gsi_arr = df['GSI'].values

                    df['FuelMoisture'] = np.where(gsi_arr >= guthresh_arr, m_arr * gsi_arr + b_arr, LFMMin)
                    df_fm = pd.DataFrame(df.FuelMoisture.groupby(df.observation_time.dt.dayofyear).mean())
                    join = samples.join(df_fm, lsuffix='_NFMD', rsuffix='_RAWS').dropna()
                    #score = 1 - spatial.distance.cosine(join.Percent_NFMD, join.Percent_RAWS)
                    score = sum(abs(join.FuelMoisture_NFMD - join.FuelMoisture_RAWS))
                    if use_prcp:
                        df_score = pd.DataFrame([combination[0], combination[1], combination[2], 
                            combination[3], combination[4], combination[5], 
                            combination[6], combination[7], score]).T 
                    else:
                        df_score = pd.DataFrame([combination[0], combination[1], combination[2], 
                            combination[3], combination[4], combination[5], score]).T 			
                    combo_list.append(df_score)
                    combo_len -= 1
                    #print(combo_len, score)
                except RuntimeWarning:
                    pass

            res = pd.concat(combo_list)
            if use_prcp:
                res.rename({0: 'TminLow', 1: 'TminUp', 2: 'VPDLow', 3: 'VPDUp', 4: 'DaylLow', 5: 'DaylUp', 6: 'PrcpRTLow', 
                    7: 'PrcpRTUp', 8: 'Score'}, axis=1, inplace=True)
            else:
                res.rename({0: 'TminLow', 1: 'TminUp', 2: 'VPDLow', 3: 'VPDUp', 4: 'DaylLow', 5: 'DaylUp', 6: 'Score'}, axis=1, inplace=True)
            res.reset_index(drop=True, inplace=True)
            res.sort_values(by='Score', inplace=True)
            return res
        res = calibrate_gsi(df_wx_daily, samples, use_vpdmax, use_prcp)
        top_res = res.head(1).copy()
        top_res.loc[:, 'LFMMax'] = LFMMax
        top_res.loc[:, 'LFMMin'] = LFMMin
        top_res.loc[:, 'GUThresh'] = GUThresh	
        top_res.loc[:, 'NwsId'] = sta_id
        top_res.loc[:, 'Area'] = analysis_data.fdop_name + ' ' +  analysis_data.fdra_name      
        if use_prcp:
            cols = ['Area', 'NwsId', 'TminLow', 'TminUp', 'VPDLow', 'VPDUp',	\
            'DaylLow', 'DaylUp', 'PrcpRTLow', 'PrcpRTUp', \
            'LFMMax', 'LFMMin', 'GUThresh', 'Score']
        else:
            cols = ['Area', 'NwsId', 'TminLow', 'TminUp', 'VPDLow', 'VPDUp',	\
            'DaylLow', 'DaylUp', 'LFMMax', 'LFMMin', 'GUThresh', 'Score']
        top_res = top_res[cols]
        top_res.loc[:, 'WxStartYr'] = st_yr
        top_res.loc[:, 'WxEndYr'] = end_yr
        top_res.loc[:, 'SampleType'] = woody_or_herb 
        
        sta_results_list.append(top_res)

        def calc_gsi_plot(df_wx_daily, use_vpdmax, use_prcp):
            df = df_wx_daily.copy()

            def Ind_Plot(Var, Low, Up):
                # Make sure all the input variables are numbers
                Var = float(Var)
                Low = float(Low)
                Up = float(Up)
                if(Up == Low):  # Upper (Up) and Lower (Low) can't be the same
                    return 0
                if( Var < Low):  # If the variables is less than the lower limit, return 0
                    return 0
                elif(Var > Up):  # If the variables is greater than the upper limit, return 1
                    return 1
                else:            # If the variables is between the lower and upper limits, return the proportion
                    return (Var - Low) / (Up - Low)

            df['TminInd'] = df.apply(lambda row: Ind_Plot(row['TminC'], Tmin[0], Tmin[1]), axis=1)
            if use_vpdmax:
                df['VPDInd'] = df.apply(lambda row: 1 - Ind_Plot(row['VPDMax'], VPD[0], VPD[1]), axis=1)
            else:
                df['VPDInd'] = df.apply(lambda row: 1 - Ind_Plot(row['VPDAvg'], VPD[0], VPD[1]), axis=1)
            df['DaylInd'] = df.apply(lambda row: Ind_Plot(row['Dayl'], Dayl[0], Dayl[1]), axis=1)
            if use_prcp:
                df['PrcpInd'] = df.apply(lambda row: Ind_Plot(row['Prcp_RT'], Prcp[0], Prcp[1]), axis=1)
                df['iGSI'] = df['TminInd'] * df['DaylInd'] * df['VPDInd'] * df['PrcpInd']
            else:
                df['iGSI'] = df['TminInd'] * df['DaylInd'] * df['VPDInd']
            df['GSI'] = df['iGSI'].rolling(28).mean()
            df['FuelMoisture'] = df.apply(lambda row: m * row.GSI + b if row.GSI >= GUThresh else LFMMin, axis=1)
            return df

        def plot_default_gsi(df_wx_daily, samples, use_vpdmax, use_prcp, ax):
            df_fm = calc_gsi_plot(df_wx_daily, use_vpdmax, use_prcp)
            df_fm.FuelMoisture.groupby(df_fm.observation_time.dt.dayofyear).mean().plot(color='grey', label="Mean", ax=ax)
            df_fm.FuelMoisture.groupby(df_fm.observation_time.dt.dayofyear).min().plot(color='#2b83ba', label="Min", ax=ax)
            df_fm.FuelMoisture.groupby(df_fm.observation_time.dt.dayofyear).max().plot(color='#d7191c', label="Max", ax=ax)
            samples.FuelMoisture.plot(color='black', linestyle='dashed', label="Sample Mean", ax=ax)
            ax.set_xlabel("Day of Year")
            ax.set_ylabel("Live Fuel Moisture (% dry wt)")
            ax.set_title("Default GSI Settings") #\n" + psa_code + " RAWS " + sta_id + " (2005-2023)")
            ax.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
            ax.text(0, samples.FuelMoisture.max(), 'Tmin: ' + str(Tmin[0]) + ', ' + str(Tmin[1]) + \
                '\nVPD: ' + str(VPD[0]) + ', ' + str(VPD[1]) + \
                '\nDayl: ' + str(Dayl[0]) + ', ' + str(Dayl[1]) + \
                '\nPrcp: ' + str(Prcp[0]) + ', ' + str(Prcp[1]) + \
                '\nUseVPDMax: ' + str(use_vpdmax) + \
                '\nUsePrecp: ' + str(use_prcp) + '\nLFMMax: ' + str(round(LFMMax, 0)) + '\nLFMMin: ' + \
                str(round(LFMMin, 0)) + '\nGUThresh: ' + str(GUThresh), fontsize=8, ha='left', va='top')
            ax.legend(loc='upper right')

        def plot_calib_gsi(df_wx_daily, samples, use_vpdmax, use_prcp, ax):
            df_fm = calc_gsi_plot(df_wx_daily, use_vpdmax, use_prcp)
            df_fm.FuelMoisture.groupby(df_fm.observation_time.dt.dayofyear).mean().plot(color='grey', label="Mean", ax=ax)
            df_fm.FuelMoisture.groupby(df_fm.observation_time.dt.dayofyear).min().plot(color='#2b83ba', label="Min", ax=ax)
            df_fm.FuelMoisture.groupby(df_fm.observation_time.dt.dayofyear).max().plot(color='#d7191c', label="Max", ax=ax)
            samples.FuelMoisture.plot(color='black', linestyle='dashed', label="Sample Mean", ax=ax)
            ax.set_xlabel("Day of Year")
            ax.set_ylabel("Live Fuel Moisture (% dry wt)")
            ax.set_title("Adjusted GSI Settings") #\n" + psa_code + " RAWS " + sta_id + " (2005-2023)")
            ax.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
            ax.text(0, samples.FuelMoisture.max(), 'Tmin: ' + str(Tmin[0]) + ', ' + str(Tmin[1]) + \
                '\nVPD: ' + str(VPD[0]) + ', ' + str(VPD[1]) + \
                '\nDayl: ' + str(Dayl[0]) + ', ' + str(Dayl[1]) + \
                '\nPrcp: ' + str(Prcp[0]) + ', ' + str(Prcp[1]) + \
                '\nUseVPDMax: ' + str(use_vpdmax) + \
                '\nUsePrecp: ' + str(use_prcp) + '\nLFMMax: ' + str(round(LFMMax, 0)) + '\nLFMMin: ' + \
                str(round(LFMMin, 0)) + '\nGUThresh: ' + str(GUThresh), fontsize=8, ha='left', va='top')
            ax.legend(loc='upper right')

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 8))

        #GSI defaults
        Tmin = (-2, 5)
        VPD = (1956, 3882)
        Dayl = (39600, 43200)
        Prcp = (0.4, 0.8)
        GUThresh = 0.3
        if herb_or_woody == 'Woody':
            LFMMax = 250
            LFMMin = 60
        else:
            LFMMax = 300
            LFMMin = 30            
        m = (LFMMax - LFMMin) / (1 - GUThresh)
        b = LFMMax - m
        plot_default_gsi(df_wx_daily, samples, use_vpdmax=use_vpdmax, use_prcp=True, ax=ax1)

        #calib GSI
        Tmin = (res.head(1).TminLow.values[0], res.head(1).TminUp.values[0])
        VPD = (res.head(1).VPDLow.values[0], res.head(1).VPDUp.values[0])
        Dayl = (res.head(1).DaylLow.values[0], res.head(1).DaylUp.values[0])
        if use_prcp:
            Prcp = (res.head(1).PrcpRTLow.values[0], res.head(1).PrcpRTUp.values[0])
        else:
            Prcp = (np.nan, np.nan)
            
        if herb_or_woody == 'Woody':
            LFMMax = 250 #samples.sample_average_value.max()
            if samples.sample_average_value.min() >= 60:
                LFMMin = samples.sample_average_value.min()
            else:
                LFMMin = 60
        else:
            LFMMax = 300 #samples.sample_average_value.max()
            if samples.sample_average_value.min() >= 30:
                LFMMin = samples.sample_average_value.min()
            else:
                LFMMin = 30   
            
        if use_prcp:
            GUThresh = 0.3
        else:
            GUThresh = 0.5
        m = (LFMMax - LFMMin) / (1 - GUThresh)
        b = LFMMax - m
        plot_calib_gsi(df_wx_daily, samples, use_vpdmax=use_vpdmax, use_prcp=use_prcp, ax=ax2)

        plt.suptitle(analysis_data.fdop_name + ' ' + analysis_data.fdra_name  + '\n' \
             + woody_or_herb + " Calibration" + "\nRAWS " + sta_id + " " + chart_yrs)
        plt.tight_layout()
        plt.savefig(output_dir + '\\calib_' + sta_id + '_' + woody_or_herb + fname_ext + '.jpg')
        plt.close()
    res = pd.concat(sta_results_list)
    res.to_csv(output_dir + '\\results_' + woody_or_herb + fname_ext + '.csv')
    return res

In [ ]:
gdf_fdra.key.sort_values().unique()

In [ ]:
#select which fdra have enough field sample data to calibrate to (review output from plot_analysis_area_fuels above)
#restart here for any other fdra in list above
gdf_select = gdf_fdra.loc[gdf_fdra.key.isin(['CentralOregon_HighDesert', 'CentralOregon_ColumbiaPlateau', 'CentralOregon_EastSlope', 'CentralOregon_Monument'])]

In [ ]:
use_prcp = False
use_vpdmax = True
woody_or_herb = 'Woody' #options are 'Woody' or 'Herbaceous'

#make analysis data, getting hourly weather for 2005 through present takes a while, so does gsi calibration...best to run overnight
for fdra_key in gdf_select.key.unique():
    gdf_select_fdra = gdf_fdra.loc[gdf_fdra.key == fdra_key]
    analysis_data = GsiAnalysisData(gdf_select_fdra, analysis_data_all)
    analysis_data.clip_data()
    analysis_data.prep_data()
    analysis_data.get_wx()
    res = calibrate_gsi(analysis_data, woody_or_herb)

In [ ]:
use_prcp = False
use_vpdmax = True
woody_or_herb = 'Woody' #options are 'Woody' or 'Herbaceous'

#use one fdra sample data to calibrate another fdra stations
def use_another():
    gdf_select_fuels = gdf_fdra.loc[gdf_fdra.key == 'CentralOregon_Monument']
    gdf_select_stations = gdf_fdra.loc[gdf_fdra.key == 'WarmSprings_FMZ2']
    
    analysis_data_stations = GsiAnalysisData(gdf_select_stations, analysis_data_all)
    analysis_data_stations.clip_data()
    analysis_data_stations.prep_data()
    analysis_data_stations.get_wx()
    
    analysis_data_stations.sites = None
    analysis_data_stations.samples_herb = None
    analysis_data_stations.samples_shrub = None
    analysis_data_stations.samples_herb_group = None
    analysis_data_stations.samples_shrub_group = None
    
    analysis_data_fuels = GsiAnalysisData(gdf_select_fuels, analysis_data_all)
    analysis_data_fuels.clip_data()
    analysis_data_fuels.prep_data()
    
    analysis_data_stations.sites = analysis_data_fuels.sites
    analysis_data_stations.prep_data()
    res = calibrate_gsi(analysis_data_stations, woody_or_herb)
use_another()    